# Introduction

Ever dealt with geographical data (including physical addresses)?

Ever wanted to get accurate coordinates of a given city, place, or even a building's address?

Ever wanted to engineer a location-related feature?

I found myself in the same situation and I end up using `geopy`, a Python interface for geocoding APIs. This notebook demonstrates my use of `geopy` to extract information from postcodes and other location-related fields.

Enjoy!

In [ ]:
import pandas as pd
full_data = pd.read_csv('../input/housing-prices-in-london/London.csv')
full_data

# What is a geocoding API?

Think Google Maps: you can send them a query string and they will search for that string and if they find a matching place/location, they'll send back a coordinate. They can also give you the more structured address of that location:

- street
- district
- state
- country
- even maybe postal codes, if available!

`geopy` provides simple, Pythonic interface for the APIs so you won't craft the requests and parse the responses by hand. One of the free and open-source geocoding APIs is Nominatim, provided by OpenStreetMap contributors.

In [ ]:
from geopy import geocoders

# Instantiate a geocoder
# Note: Nominatim requires user_agent field to be filled with
# a general idea of what you are using Nominatim for.
geocoder = geocoders.Nominatim(user_agent = 'kaggle data science london housing price')

In [ ]:
# Use the geocoder to send a sample request,
# plus additional address details.
result = geocoder.geocode('London', addressdetails = True)
result

# The response object

The following cells demonstrate what can be accessed from the `geopy`'s resulting Location object.

In [ ]:
result.point

In [ ]:
(result.latitude, result.longitude, result.altitude)

In [ ]:
result.raw

# The 'address' field is our juicy information.

`result.raw['address']` above will return a dictionary that enlists a very neatly structured address hierarchy. If we use it with specific enough place, we can even get the name of the building, street, and the surrounding suburbs!

In [ ]:
# Search for Hornsey Town Hall within Great Britain only.
result = geocoder.geocode('Hornsey Town Hall', addressdetails = True, country_codes = 'gb')
result

# Processing and rate-limiting

We can make the geocoding calls more convenient by making a helper function that

- process our inputs by adding arguments that we'll always use, and
- process the outputs by giving only fields that we want.

Furthermore, we can rate-limit our requests so we don't overwhelm the server.

In [ ]:
# This is a helper function to screen in only these fields
cols = ['suburb', 'town', 'city', 'state_district', 'state', 'country']
def geocode(query):
    # Make a geocoding request with query,
    # restrict the results to Great Britain,
    # and give detailed address structure.
    result = geocoder.geocode(query, country_codes = 'gb', addressdetails = True)
    
    # If there is no result with that query, return None
    if result is None:
        return None
    
    # If there is a result, return only the desired fields
    address = result.raw['address']
    address = {key: value for key, value in address.items() if key in cols}
    return address

# Rate limit the requests to only 20 requests per second max.
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geocode, min_delay_seconds = 1/20)

In [ ]:
# Call our function above with a postcode
addr = geocode('SW19 8NY')
addr

In [ ]:
# Call our function above with a street name plus city
addr = geocode('Seward Street, London')
addr

In [ ]:
# Call our function with a place that definitely does not exist
# in Great Britain
addr = geocode('Uttar Pradesh')
addr is None

In [ ]:
# Call our function with a typo
addr = geocode('Birminghm')
addr is None

In [ ]:
pd.json_normalize(full_data["Postal Code"].sample(10).apply(geocode))

In [ ]:
%%time

# Get a fully structured dataframe of address by searching for UK postcodes
# in Nominatim's OpeenStreetMap database.
nominatim_address = full_data['Postal Code'].apply(geocode)

# Convert the Series of dict into proper dataframe
nominatim_address = pd.json_normalize(nominatim_address)

# Append the original Postal Code to the left
nominatim_address = pd.concat([full_data['Postal Code'], nominatim_address], axis = 1)

# Display
nominatim_address

Some caveat:

- Nominatim is an open-source, free APIs, provided by community contributions and donations. Being free, it rate-limits your use -- the above cell takes about ~30 minutes for ~3500 entries or about 2 entries per second. You are also not allowed to do bulk-geocoding and scraping, it violates the terms of service.

- Nominatim's database (OpenStreetMap) is also powered by community contributions. In my experience, when I try to use Nominatim on rural or remote places, the results get less detailed -- maybe because there's less contributors for that area. However, it's still a very capable free alternative for research and experiment purposes.

- Nominatim does not handle typos! You get what you type in! This is good, actually; so you won't get coordinates of a wrong place halfway across the world due to mistyping on you or your data's end.

# Closing

That's it! I hope this notebook helps you in dealing with addresses and other location-related data by using free, open-source, geocoding APIs. Being a citizen of the internet that loves freedom in research, you know I love free things ;)

Feel free to upvote and fork if you like to try this yourself. Keep learning and happy data-sciencing!